In [2]:
import os

In [3]:
%pwd

'd:\\text_summ_krish_naik\\Text-Summarizer-Project\\research'

In [4]:
os.chdir('../')

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [6]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories

Logging module initialized!


In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.evaluation_strategy,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

        return model_trainer_config

In [6]:
import torch

In [8]:
import torch
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk


d:\text_summ_krish_naik\Text-Summarizer-Project\textS\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-04-05 22:11:44,187: INFO: config: PyTorch version 2.6.0 available.]


In [11]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
        
        #loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        # trainer_args = TrainingArguments(
        #     output_dir=self.config.root_dir, num_train_epochs=self.config.num_train_epochs, warmup_steps=self.config.warmup_steps,
        #     per_device_train_batch_size=self.config.per_device_train_batch_size, per_device_eval_batch_size=self.config.per_device_train_batch_size,
        #     weight_decay=self.config.weight_decay, logging_steps=self.config.logging_steps,
        #     evaluation_strategy=self.config.evaluation_strategy, eval_steps=self.config.eval_steps, save_steps=1e6,
        #     gradient_accumulation_steps=self.config.gradient_accumulation_steps
        # ) 


        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=500,
            per_device_train_batch_size=1, per_device_eval_batch_size=1,
            weight_decay=0.01, logging_steps=10,
            evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
            gradient_accumulation_steps=16
        ) 

        trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["train"], 
                  eval_dataset=dataset_samsum_pt["validation"])
        
        trainer.train()

        ## Save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        ## Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))

In [13]:
import os

model_dir = "artifacts/model_trainer/"

if not os.path.exists(model_dir):
    print(f"🚨 Directory '{model_dir}' does not exist!")
elif not os.listdir(model_dir):  
    print(f"🚨 No model files found in '{model_dir}'")
else:
    print(f"✅ Model files found: {os.listdir(model_dir)}")


🚨 No model files found in 'artifacts/model_trainer/'


In [ ]:
model.save_pretrained("artifacts/model_trainer/")


NameError: name 'model' is not defined

In [14]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()

    # Check if model_trainer_config is correctly initialized
    print("Model Trainer Config:", model_trainer_config)  # Debugging

    # Ensure model_trainer_config is an instance of ModelTrainerConfig
    if not isinstance(model_trainer_config, ModelTrainerConfig):
        raise TypeError("❌ model_trainer_config is not an instance of ModelTrainerConfig")

    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    print(f"🚨 Error: {e}")


[2025-04-05 22:13:06,892: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-05 22:13:06,900: INFO: common: yaml file: config\params.yaml loaded successfully]
[2025-04-05 22:13:06,904: INFO: common: created directory at: artifacts]
[2025-04-05 22:13:06,906: INFO: common: created directory at: artifacts/model_trainer]
Model Trainer Config: ModelTrainerConfig(root_dir='artifacts/model_trainer', data_path='artifacts/data_transformation/samsung_dataset', model_ckpt='artifacts/model_trainer/checkpoint.pth', num_train_epochs=1, warmup_steps=500, per_device_train_batch_size=1, weight_decay=0.01, logging_steps=10, evaluation_strategy='steps', eval_steps='steps', save_steps='1e6', gradient_accumulation_steps=16)
🚨 Error: Repo id must be in the form 'repo_name' or 'namespace/repo_name': 'artifacts/model_trainer/checkpoint.pth'. Use `repo_type` argument if needed.


In [21]:
from transformers import AutoModel

model = AutoModel.from_pretrained("artifacts/model_trainer/", local_files_only=True)


ValueError: Unrecognized model in artifacts/model_trainer/. Should have a `model_type` key in its config.json, or contain one of the following strings in its name: albert, align, altclip, aria, aria_text, audio-spectrogram-transformer, autoformer, aya_vision, bamba, bark, bart, beit, bert, bert-generation, big_bird, bigbird_pegasus, biogpt, bit, blenderbot, blenderbot-small, blip, blip-2, bloom, bridgetower, bros, camembert, canine, chameleon, chinese_clip, chinese_clip_vision_model, clap, clip, clip_text_model, clip_vision_model, clipseg, clvp, code_llama, codegen, cohere, cohere2, colpali, conditional_detr, convbert, convnext, convnextv2, cpmant, ctrl, cvt, dab-detr, dac, data2vec-audio, data2vec-text, data2vec-vision, dbrx, deberta, deberta-v2, decision_transformer, deformable_detr, deit, depth_anything, depth_pro, deta, detr, diffllama, dinat, dinov2, dinov2_with_registers, distilbert, donut-swin, dpr, dpt, efficientformer, efficientnet, electra, emu3, encodec, encoder-decoder, ern